In [ ]:
import glob
import numpy as np
import pandas as pd
import os
import shutil 
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
import keras
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
%matplotlib inline

In [ ]:
# Define data path
PATH = os.getcwd()
data_path = PATH + '/CT'
data_dir_list = os.listdir(data_path)

In [ ]:
## Image processing ##
IMG_WIDTH=224
IMG_HEIGHT=224
IMG_DIM = (IMG_WIDTH, IMG_HEIGHT)
num_channel=1
num_classes = 2
labels_name={'Covid':0,'non-Covid':1}

img_data_list=[]
labels_list = []


for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    label = labels_name[dataset]
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(IMG_WIDTH,IMG_HEIGHT))
        img_data_list.append(input_img_resize)
        labels_list.append(label)


In [ ]:
## Image normalization ##
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
print (img_data.shape)

In [ ]:
## Label one hot encoding ##
labels = np.array(labels_list)
print(np.unique(labels,return_counts=True))
Y = np_utils.to_categorical(labels, num_classes)

In [ ]:
#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=4)

In [ ]:
## Initialization of pre-trained ResNet50
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import AveragePooling2D

# include top should be False to remove the softmax layer
pretrained_model = ResNet50(include_top=False, weights='imagenet')

## Output based on average_pooling2d_1 ##
output = pretrained_model.layers[-33].output
output = keras.layers.AveragePooling2D((1,1))(output)
pretrained_model = keras.Model(pretrained_model.input, output)
pretrained_model.summary()

In [ ]:
## Features extraction ##
from keras.utils import to_categorical

vgg_features_train = pretrained_model.predict(X_train) #train features
vgg_features_val = pretrained_model.predict(X_test)    #val features


In [ ]:
### Fully Connected Layers ###
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout, GlobalAveragePooling2D

model2 = Sequential()
model2.add(Flatten(input_shape=(14, 14, 1024)))
model2.add(Dense(100, activation='relu'))
model2.add(BatchNormalization(momentum=0.8))
model2.add(Dropout(0.5))
model2.add(Dense(2, activation='sigmoid'))

# compile the model
model2.compile(optimizer=keras.optimizers.adam(lr=0.0003, decay= 1e-5) , metrics=['accuracy'], loss='binary_crossentropy')

model2.summary()

# anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
# checkpoint = ModelCheckpoint('model/ResNet_Classifier.h5', verbose=1, save_best_only=True)
#callbacks=[anne, checkpoint]
# train model using features generated from VGG16 model

hist = model2.fit(vgg_features_train, y_train, epochs=100, batch_size=128, validation_data=(vgg_features_val, y_test))
#model2.save('model/ResNet_Classifier_AveragePooling2D.h5')

In [ ]:
## Saving training acc and loss to CSV
hist_df = pd.DataFrame(hist.history) 

hist_csv_file = 'history/Resnet_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
    
history = pd.read_csv("history/Resnet_history.csv")
val_loss = history['val_loss'].to_numpy()
val_acc = history['val_accuracy'].to_numpy()
loss = history['loss'].to_numpy()
acc = history['accuracy'].to_numpy()

In [ ]:
## PLotting acc vs loss ##
plt.plot(acc)
plt.plot(loss)
plt.plot(val_acc)
plt.plot(val_loss)
plt.title('Training Accuracy', fontsize=20)
plt.yticks(fontsize=20)
plt.ylabel('Accuracy (%)', fontsize=20)
plt.xticks(fontsize=20)
plt.xlabel('Epoch',fontsize=20)
plt.legend(['Train_acc', 'Train_loss', 'Val_acc', 'Val_loss'], loc='best', fontsize=22)
plt.gcf().set_size_inches(15,10) 
plt.savefig('C:/Users/CooL/Desktop/Result/VGG/Resnet_train_graph.jpg')
plt.show()

In [ ]:
PATH = os.getcwd()
# Define data path
data_path = PATH + '/Testing_Set'
data_dir_list = os.listdir(data_path)

import seaborn as sns
import matplotlib.pyplot as plt   
from sklearn.metrics import confusion_matrix, roc_curve

IMG_WIDTH=224
IMG_HEIGHT=224
test_list = []
y_pred = []
labels_name ={'Covid':0,'non-Covid':1}


for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    label = labels_name[dataset]
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(IMG_WIDTH,IMG_HEIGHT))
        test_list.append(input_img_resize)
        y_pred.append(label)

test_image = np.array(test_list)
test_image = test_image.astype('float32')
test_image /= 255

test_features = pretrained_model.predict(test_image)


predict_list = []
predict_1 = model2.predict_proba(test_features)

fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_pred, predict_1[:,1])

from sklearn.metrics import auc
import pandas as pd
auc_keras = auc(fpr_keras, tpr_keras)
optimum = thresholds_keras[np.argmax(tpr_keras - fpr_keras)]
print('Optimal cut off point = ',optimum)
print('AUC = ', auc_keras)

gmeans = np.sqrt(tpr_keras * (1-fpr_keras))
ix = np.argmax(gmeans)
optimal = thresholds_keras[ix]
print('Optimal cut off point = ',thresholds_keras[ix])


x = optimal # Best => x = 0.98892

for j in range(0,1):
    for i in range(0, len(predict_1)):
        if predict_1[i][1] >= x:
            predict_list.append(1)
        else:
            predict_list.append(0)
    print(confusion_matrix(y_pred, predict_list), 'x =',"%.5f" % x, '\n')

cf_matrix=confusion_matrix(y_pred, predict_list)
tp = cf_matrix[0][0]
fn = cf_matrix[0][1]
fp = cf_matrix[1][0]
tn = cf_matrix[1][1]

TrueP = '{0:.2%}'.format(tp/(tp+fn))
FalseN = '{0:.2%}'.format(fn/(tp+fn))
FalseP = '{0:.2%}'.format(fp/(fp+tn))
TrueN = '{0:.2%}'.format(tn/(fp+tn))
print(TrueP, FalseN, FalseP, TrueN)

xy_Labels = ['Covid', 'non-Covid']
group_names = ['True Positive', 'False Negative', 'False Positive','True Negative']
group_counts = ['{0:0.0f}'.format(value) for value in cf_matrix.flatten()]
group_percentage = [TrueP, FalseN, FalseP, TrueN]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentage)]
labels = np.asarray(labels).reshape(2,2)


### Confusion Matrix of ResNet50 ###
# plt.title('(ResNet-50), Threshold = '+str("{:.4f}".format(optimal)), fontsize = 20)
plt.title('Confusion Matrix (ResNet-50)',fontsize = 18)
sns.heatmap(cf_matrix, fmt='', annot=labels, annot_kws={'size':18} ,xticklabels=xy_Labels, yticklabels=xy_Labels, cmap='Blues')
plt.xlabel('Predicted label', fontsize = 15) # x-axis label with fontsize 15
plt.ylabel('True label', fontsize = 15) # y-axis label with fontsize 15
#plt.savefig('C:/Users/CooL/Desktop/Result/Resnet/Resnet_CF.jpg')
plt.show()

### ROC Curves ResNet50 ###
plt.figure(figsize=(6,6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.scatter(fpr_keras[ix],tpr_keras[ix], marker='o', color='black', label='Optimal Threshold = {:.4f})'.format(optimal))
plt.xlabel('False positive rate (FPR)', fontsize=15)
plt.ylabel('True positive rate (TPR)', fontsize=15)
plt.title('ROC (ResNet-50)',fontsize=22)
plt.legend(loc='best', fontsize=15)
#plt.savefig('C:/Users/CooL/Desktop/Result/Resnet/Resnet_graph.jpg')
plt.show()


